In [8]:
from functions.satellites_ephemeris import SatelliteEphemeris

filename = "functions/ephemerides/tles50.json"
ephemeris = SatelliteEphemeris()
ephemeris.load_data()
debris_data = ephemeris.get_debris()

print(f"Number of objects in DB: {len(debris_data)} \n")

GTOC9 database loaded.
Number of objects in DB: 123 



In [16]:
# from functions.satellites_ephemeris import SatelliteEphemeris

# filename = 'functions/ephemerides/OneWeb_Sat_Data.xlsx'
# ephemeris = SatelliteEphemeris(filename=filename)
# ephemeris.load_data()
# debris_data = ephemeris.get_debris()

Custom database loaded.


In [32]:
from functions.MRPLP_J2_analytic import MultiRevolutionPerturbedLambertSolver
import numpy as np

t1 = 23902.49
id1 = '45'

t2 = 23919.56
id2 = '82'

cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )

tof = ( t2 - t1 ) * 86400

mu = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

MRPLPsolver = MultiRevolutionPerturbedLambertSolver() # MRPLP solver

# set the parameters for the MRPLP solver
order = 7
parameters = MRPLPsolver.mrplp_J2_analytic_parameters(cart1[0:3], cart2[0:3], tof, cart1[3:], mu, Re, J2,
                                            order, tol=1.0e-6, cont=0.0, dcontMin=0.1, scl=1.0e-3, itermax=200 )

# solve the MRPLP
output = MRPLPsolver.mrplp_J2_analytic(parameters)

# compute the DV
dvv1 = output.vv1Sol - cart1[3:]
dvv2 = cart2[3:] - output.vv2Sol
dv1 = np.linalg.norm( dvv1 )
dv2 = np.linalg.norm( dvv2 )
dvtot = dv1 + dv2

# print the output
print(f"-------------------------------------------------------")
print(f"                 OUTPUT SUMMARY")
print(f"-------------------------------------------------------")
print(f"Order of the expansion  : {order}")
print(f"Success                 : {output.success}")
print(f"Elapsed time            : {output.elapsed_time} seconds")
print(f"Final pos. error (norm) : {np.linalg.norm( output.rr2DA - cart2[0:3] )} m")
print(f"-------------------------------------------------------")
print(f"Delta_v1                : {dv1} m/s")
print(f"Delta_v2                : {dv2} m/s")
print(f"Delta_vtot              : {dvtot} m/s")
print(f"-------------------------------------------------------")


-------------------------------------------------------
                 OUTPUT SUMMARY
-------------------------------------------------------
Order of the expansion  : 7
Success                 : True
Elapsed time            : 0.6622529029846191 seconds
Final pos. error (norm) : 6.135841706781253e-06 m
-------------------------------------------------------
Delta_v1                : 217.1656018203001 m/s
Delta_v2                : 128.2283888671467 m/s
Delta_vtot              : 345.39399068744683 m/s
-------------------------------------------------------


In [19]:
from functions.analytic_approx_dv_j2 import AnalyticDVApproxJ2

start = 8487.0
end = start + 30
step = 1.0
t1_list = np.arange(t1, end + step, step)

id_start = [str(i) for i in range(1, len(debris_data))]
id_tar = [str(i) for i in range(1, len(debris_data))]

########## START: define the DV options ##########
dv_tol_an = 1500
dv_tol = 200
########## END: define the DV options ##########

########## START: define the TOF options ##########
start_tof = 5.0 # days
end_tof = 30.0 # days
grid_size = 2.0 # days
tof_list = np.arange(start_tof, end_tof + grid_size, grid_size)
########## END: define the TOF options ##########

for id1 in id_start:

    for t1 in t1_list:

        for id2 in id_tar:

            if id1 != id2:

                for tof in tof_list:

                    t2 = t1 + tof
                    tof_secs = tof * 86400.0

                    cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
                    cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )
                    vv1g = cart1[3:]

                    # maybe first do an approximate DV computation, and then use the MRPLP
                    paramsAnalytical = AnalyticDVApproxJ2.approx_DV_parameters( cart1[0:3], cart1[3:],
                                                                                cart2[0:3], cart2[3:],
                                                                                t1, t2,
                                                                                mu, Re, J2)
                    delta_v = AnalyticDVApproxJ2.approx_DV(paramsAnalytical)

                    if delta_v <= dv_tol_an:

                        # set the parameters for the MRPLP solver
                        order = 7
                        parameters = MRPLPsolver.mrplp_J2_analytic_parameters(cart1[0:3], cart2[0:3], tof_secs, vv1g, mu, Re, J2,
                                                                            order, tol=1.0e-6, cont=0.0, dcontMin=0.1, 
                                                                            scl=1.0e-3, itermax=150 )
                        
                        # solve the MRPLP
                        output = MRPLPsolver.mrplp_J2_analytic(parameters)
                        
                        # compute the DV
                        dvv1 = output.vv1Sol - cart1[3:]
                        dvv2 = cart2[3:] - output.vv2Sol
                        dv1 = np.linalg.norm( dvv1 )
                        dv2 = np.linalg.norm( dvv2 )
                        dvtot = dv1 + dv2

                        # print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, DV: {delta_v} m/s - approx. DV")
                        if np.isnan(dvtot):
                            """ print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION - NaN") """
                        else:
                            if dvtot < dv_tol:
                                print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, DV: {dvtot} m/s")
                            else:
                                """ print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION") """
                    else:
                        """ print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION - approx. DV: {delta_v} m/s") """


(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8494.72551337956)), tof: 7.0 days, DV: 27.42090499676192 m/s
(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8496.72551337956)), tof: 9.0 days, DV: 50.430137283381995 m/s
(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8500.72551337956)), tof: 13.0 days, DV: 31.49152130934543 m/s
(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8502.72551337956)), tof: 15.0 days, DV: 20.615629948338057 m/s
(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8506.72551337956)), tof: 19.0 days, DV: 22.936374403976053 m/s
(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8508.72551337956)), tof: 21.0 days, DV: 11.673359914163779 m/s
(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8512.72551337956)), tof: 25.0 days, DV: 13.629683403946274 m/s
(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8514.72551337956)), tof: 27.0 days, DV: 7.0719921686708185 m/s
(ID1, t1)=(('1', 8487.72551337956))--> ID2=(('2', 8518.72551337956)), tof: 31.0 days, DV: 7.92615640

KeyboardInterrupt: 

In [16]:
# propagate and obtain the STM
states, vecttof, STM = (
    MultiRevolutionPerturbedLambertSolver.propagatePerturbedKeplerSTM(
        rr1, vv1Sol, tof, 1000.0, parameters
    )
)

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Extract x, y, z components
x = states[:, 0]
y = states[:, 1]
z = states[:, 2]

# Create interactive 3D plot
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='lines',
    line=dict(width=2),
    name='trajectory'
)])

# Add initial and final points
fig.add_trace(go.Scatter3d(
    x=[x[0]],
    y=[y[0]],
    z=[z[0]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[1]],
    y=[y[1]],
    z=[z[1]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[2]],
    y=[y[2]],
    z=[z[2]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[3]],
    y=[y[3]],
    z=[z[3]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[-1]],
    y=[y[-1]],
    z=[z[-1]],
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Final Point'
))

# Labels and title
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# Show the plot
fig.show()

In [ ]:
# analytic computation of DV
from functions.analytic_approx_dv_j2 import AnalyticDVApproxJ2
import numpy as np


mue = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

id1 = '0'
id2 = '302'
t1 = 8487.0
t2  = t1 + 10.0

cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )

# Initialize parameters
parameters = AnalyticDVApproxJ2.approx_DV_parameters(
    rr1=cart1[0:3],
    vv1=cart1[3:],
    rr2=cart2[0:3],
    vv2=cart2[3:],
    t1=t1,
    t2=t2,
    mu=mue,
    rE=Re,
    J2=J2
)

delta_v = AnalyticDVApproxJ2.approx_DV(parameters)
delta_v